# Full Text Search

This is a Python port of a golang implementation from [here](https://artem.krylysov.com/blog/2020/07/28/lets-build-a-full-text-search-engine/).



## Corpus

The [latest dump](https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract1.xml.gz) of abstract of English Wikipedia from dumps.wikimedia.org. The file size after decompression is 913 MB. The XML file contains over 600K documents.

In [1]:
import gzip
import shutil
from pathlib import Path
from tqdm import tqdm
import urllib.request

CORPUS_URL = "https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract1.xml.gz"
zip_file = Path(CORPUS_URL.split("/")[-1])
xml_file = zip_file.name.replace(".gz", "")

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)
        
if not zip_file.exists():
    print("Downloading data file(zipped).")
    download_url(CORPUS_URL, zip_file.name)
else:
    print("Data file(zipped) already exists.")

with gzip.open(zip_file, "rb") as f_in:
    with open(xml_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        print(f"Extracted {zip_file.name}")

Data file(zipped) already exists.
Extracted enwiki-latest-abstract1.xml.gz


## Load documents

In [2]:
from timeit import default_timer as timer

class Benchmark:
    """
    Helper context manager to profile time taken.
    """
    def __init__(self, msg):
        self.msg = msg

    def __enter__(self):
        self.start = timer()
        return self

    def __exit__(self, *args):
        t = timer() - self.start
        print(f"{self.msg} : {t:.2f} seconds")
        self.time = t

In [3]:
import xml.etree.ElementTree as xml

def load_documents(path):
    tree = xml.parse(path)
    root = tree.getroot()
    docs = []
    for index, xml_doc in enumerate(root):
        doc = {"id": index}
        for elm in xml_doc:
            if elm.tag not in ['title', 'abstract', 'url']:
                continue
            doc[elm.tag] = elm.text or ""
        docs.append(doc)
    return docs

with Benchmark("Loading docs"):
    docs = load_documents(xml_file)
    
print(docs[0])

Loading docs : 25.38 seconds
{'id': 0, 'title': 'Wikipedia: Anarchism', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'abstract': 'Anarchism is a political philosophy and movement that rejects all involuntary, coercive forms of hierarchy. It calls for the abolition of the state which it holds to be undesirable, unnecessary, and harmful.'}


## Naive search

In [4]:
def naive_search(docs, term):
    results = []
    for doc in docs:
        if doc.get("abstract") and term in doc["abstract"]:
            results.append(doc)
    return results

with Benchmark("Naive searching cat"):
    results = naive_search(docs, "Cat")

Naive searching cat : 0.21 seconds


In [5]:
# TODO
import re

def regex_search(docs, term):
    results = []
    for doc in docs:
        re.compile()
        results.append(doc)
    return results
#     re := regexp.MustCompile(`(?i)\b` + term + `\b`)
#         if re.MatchString(doc.Text) {



## Tokenize

In [6]:
def tokenize(text):
    pattern = re.compile("\w+")
    matches = pattern.finditer(text)
    return [text[match.start():match.end()] for match in matches]
    # TODO: Without finditer?
    # return strings.FieldsFunc(text, func(r rune) bool {
    #     // Split on any character that is not a letter or a number.
    #     return !unicode.IsLetter(r) && !unicode.IsNumber(r)
    # })


In [7]:
def lower(tokens):
    return [token.lower() for token in tokens]

In [8]:
# Top 10 from OEC
# https://en.wikipedia.org/wiki/Most_common_words_in_English
STOP_WORDS = set(["a", "and", "be", "have", "i", "in", "of", "that", "the", "to"])

def remove_stopwords(tokens):
    return [token for token in tokens if token not in STOP_WORDS]
    pass


## Stemming

https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python
 - Porter Stemmer
 - Snowball Stemmers
 - What is lemmatization?

In [9]:
from nltk.stem.snowball import SnowballStemmer

def reduce_to_stem(tokens):
    stemmer = SnowballStemmer(language="english")
    return [stemmer.stem(token) for token in tokens]

## Put all the steps together


In [10]:
def analyse(text):
    tokens = tokenize(text)
    tokens = lower(tokens)
    tokens = remove_stopwords(tokens)
    tokens = reduce_to_stem(tokens)
    return tokens

## Building the index

In [11]:
from typing import Dict, List, Union

class BaseIndex:
    def __init__(self):
        self.index: Dict[str, List[int]] = {}
        
    def add(self, doc: Dict[str, Union[int, str]]):
        id_ = doc["id"]
        text = doc["text"]
            
        for token in analyse(doc["text"]):
            ids = self.index.get(token, [])
            if id_ not in ids:
                # TODO: Ensure ascending order of Ids
                ids.append(id_)
                self.index[token] = ids
            
search_index = BaseIndex()
search_index.add({"id": 1, "text": "A donut on a glass plate. Only the donuts."})
search_index.add({"id": 2, "text": "donut is a donut"})
print(search_index.index)

{'donut': [1, 2], 'on': [1], 'glass': [1], 'plate': [1], 'onli': [1], 'is': [2]}


## Querying

In [12]:
class IndexWithSearch(BaseIndex):
    """Extends the BaseIndex with search functionality."""
    
    def search(self, text):
        results = []
        for token in analyse(text):
            results.extend(self.index.get(token, []))
        return results

In [13]:
import xml.etree.ElementTree as xml

def index_documents_from_xml(path, search_index, limit=None):
    """Load documents from the XML and add them to the index."""
    
    tree = xml.parse(path)
    root = tree.getroot()
    for id_, xml_doc in enumerate(root):
        # Adding first `limit` docs from corpus to the index
        if limit:
            if id_>0 and id_%100==0:
                break
                
        doc = {"id": id_}
        for elm in xml_doc:
            # Ignore all other fields except abstract.
            if elm.tag not in ['abstract']:
                continue
            abstract = elm.text or ""
            search_index.add({"id": id_, "text": abstract})            
    return

# Creating an index with all the abstracts in the corpus
search_index = IndexWithSearch()
with Benchmark("Indexing abstract from XML"):
    index_documents_from_xml(xml_file, search_index, limit=100)
    

Indexing abstract from XML : 37.08 seconds


In [14]:
search_index.search("solar")

[2]

## Boolean Queries

In [15]:
from typing import Set

class IndexWithBoolean(BaseIndex):
    """Extends the BaseIndex with search and boolean queries."""
    
    def __init__(self):
        # Change to set instead of list so we can do intersection
        self.index: Dict[str, Set[int]] = {}
        
    def add(self, doc: Dict[str, Union[int, str]]):
        id_ = doc["id"]
        text = doc["text"]
        for token in analyse(doc["text"]):
            ids = self.index.get(token, set())
            if id_ not in ids:
                ids.add(id_)
                self.index[token] = ids
                
    def search(self, text):
        results = set()
        for token in analyse(text):
            term_results = self.index.get(token, set())
            if not results:
                results = term_results
            else:
                results = results.intersection(term_results)
        
        return list(results)
    
    
search_index = IndexWithBoolean()
with Benchmark("Indexing abstract from XML"):
    index_documents_from_xml(xml_file, search_index, limit=100)

Indexing abstract from XML : 26.47 seconds


In [16]:
with Benchmark("Indexing search"):
    search_index.search("is black")

Indexing search : 0.00 seconds


## Future

- Extend boolean queries to support OR and NOT.
- Store the index on disk:
   - Rebuilding the index on every application restart may take a while.
   - Large indexes may not fit in memory.

- Experiment with memory and CPU-efficient data formats for storing sets of document IDs. Take a look at [Roaring Bitmaps](roaringbitmap.org).
  See [blevesearch(golang)](https://blevesearch.com/).
- Support indexing multiple document fields.
- Sort results by relevance.

- [TFIDF](https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.115.8343)
- BM25
- Soundex for phonetic spelling errors
- Stop word libs